O objetivo do código aqui é realizar manipulações nos dados.

Manipulações realizadas:

1. $\Delta x_t$

2. $\Delta^2 x_t$

3. $ln(x_t)$

4. $\Delta ln(x_t)$

5. $\Delta^2 ln(x_t)$

In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("dados/dados_crus2003.csv", sep=";", decimal=",", thousands=".")
df.head(5)

,month,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
0,jan/03,2.25,1.98,25.145161,94977783.0,391237088.0,696054502.0,810272451.0,10941.09,3.4384,...,48763.52,0.9,88.150920,185.591626,103.0,14.068293,NaN,250.14,239.98,203.34
1,fev/03,1.57,2.19,25.821429,93602937.0,393976058.0,706788435.0,815745074.0,10280.61,3.5908,...,51066.28,0.6,120.281038,161.759767,103.5,14.570732,NaN,250.13,233.46,206.19
2,mar/03,1.23,1.14,26.500000,89822584.0,388142457.0,710712013.0,820859498.0,11273.63,3.4469,...,51462.26,0.4,133.688711,224.647203,101.9,15.198780,182585.2573,248.06,234.18,202.43
3,abr/03,0.97,1.14,26.500000,87410599.0,385467653.0,712800120.0,823489095.0,12556.70,3.1187,...,55840.39,0.0,96.791932,176.579867,107.7,15.575610,178958.1301,247.57,235.03,204.02
4,mai/03,0.61,0.85,26.500000,86231763.0,388490690.0,721321334.0,833412205.0,13421.60,2.9557,...,57077.43,-0.3,85.230528,122.309831,112.0,16.203659,177284.6602,250.48,238.04,206.54


In [17]:
# Cria dicionário de listas vazias com nomes de colunas
delta_dict = {}
for col_name, _ in df.items():
    if col_name == "month":
        continue
    delta_dict[col_name] = []

# Popula novo DataFrame com a diferença dos dados no primeiro
for i, row in df.iterrows():
    # Pula primeira linha
    if i == 0:
        continue
    for col_name, value in row.items():
        # Ignora coluna de meses
        if col_name == "month":
            continue
        # Salva diferenças em dicionário
        delta_dict[col_name].append(df.loc[i, col_name] - df.loc[i - 1, col_name])

delta_df = pd.DataFrame(delta_dict)
delta_df.head(5)

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
0,-0.68,0.21,0.676267,-1374846.0,2738970.0,10733933.0,5472623.0,-660.48,0.1524,2.933,...,2302.76,-0.3,32.130118,-23.831859,0.5,0.502439,NaN,-0.01,-6.52,2.85
1,-0.34,-1.05,0.678571,-3780353.0,-5833601.0,3923578.0,5114424.0,993.02,-0.1439,-5.216,...,395.98,-0.2,13.407673,62.887436,-1.6,0.628049,NaN,-2.07,0.72,-3.76
2,-0.26,0.00,0.000000,-2411985.0,-2674804.0,2088107.0,2629597.0,1283.07,-0.3282,-10.233,...,4378.13,-0.4,-36.896779,-48.067337,5.8,0.376829,-3627.1272,-0.49,0.85,1.59
3,-0.36,-0.29,0.000000,-1178836.0,3023037.0,8521214.0,9923110.0,864.90,-0.1630,-1.510,...,1237.04,-0.3,-11.561404,-54.270036,4.3,0.628049,-1673.4699,2.91,3.01,2.52
4,-0.76,-0.63,-0.183333,2234552.0,-2152663.0,10208771.0,6985494.0,-449.02,-0.0725,-0.308,...,-2498.83,0.1,-28.352196,-11.614289,4.8,0.125610,-1652.8386,-1.42,-3.46,-0.18


In [15]:
delta_df.tail(5)    

{'month': [], 'ipca': [-0.6799999999999999, -0.3400000000000001, -0.26, -0.36, -0.76, 0.35, 0.14, 0.44, -0.49000000000000005, 0.050000000000000044, 0.18, 0.24, -0.15000000000000002, -0.14, -0.09999999999999998, 0.14, 0.19999999999999996, 0.20000000000000007, -0.22000000000000008, -0.35999999999999993, 0.10999999999999999, 0.24999999999999994, 0.17000000000000004, -0.28, 0.010000000000000009, 0.020000000000000018, 0.26, -0.38, -0.51, 0.27, -0.07999999999999999, 0.17999999999999997, 0.4, -0.19999999999999996, -0.19000000000000006, 0.22999999999999998, -0.18, 0.020000000000000018, -0.22, -0.10999999999999999, -0.31, 0.4, -0.14, 0.15999999999999998, 0.12000000000000002, -0.020000000000000018, 0.16999999999999998, -0.03999999999999998, 0.0, -0.07, -0.12, 0.030000000000000027, 0.0, -0.040000000000000036, 0.22999999999999998, -0.29, 0.12, 0.08000000000000002, 0.36, -0.19999999999999996, -0.050000000000000044, -0.010000000000000009, 0.07000000000000006, 0.24, -0.050000000000000044, -0.20999999